In [1]:
from FPTU_FA24_EEG_Artifacts_Recognition.config import *
from FPTU_FA24_EEG_Artifacts_Recognition.pipeline import *

CONFIG_FILE_PATH = Path(r'D:\Dang_project\FPTU_FA24_EEG_Artifacts_Recognition\config\config.yaml')
PARAMS_FILE_PATH = Path(r'D:\Dang_project\FPTU_FA24_EEG_Artifacts_Recognition\config\params.yaml')
DATASET_DETAIL_FILE_PATH = Path(r'D:\Dang_project\FPTU_FA24_EEG_Artifacts_Recognition\config\dataset.yaml')

config_manager = ConfigurationManager(
    CONFIG_FILE_PATH,
    PARAMS_FILE_PATH,
    DATASET_DETAIL_FILE_PATH
)

d:\Dang_project\FPTU_FA24_EEG_Artifacts_Recognition\tests
2024-11-19 13:45:00,962: INFO: config: TensorFlow version 2.18.0 available.


d:\Dang_project\FPTU_FA24_EEG_Artifacts_Recognition\thesis\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2024-11-19 13:45:03,441: INFO: common: yaml file 'D:\Dang_project\FPTU_FA24_EEG_Artifacts_Recognition\config\config.yaml' loaded successfully
2024-11-19 13:45:03,448: INFO: common: yaml file 'D:\Dang_project\FPTU_FA24_EEG_Artifacts_Recognition\config\params.yaml' loaded successfully
2024-11-19 13:45:03,448: INFO: common: yaml file 'D:\Dang_project\FPTU_FA24_EEG_Artifacts_Recognition\config\dataset.yaml' loaded successfully


In [5]:
path = Path(os.path.join(
    dataset_config.output_data_path, 'left'
))

In [15]:
ds = datasets.Dataset.load_from_disk(r'D:\Dang_project\FPTU_FA24_EEG_Artifacts_Recognition\artifacts\transform_data\left')
ds.set_format('tf')
ds['left'].shape, ds['right'].shape, ds['both'].shape

(TensorShape([1834, 64, 4]),
 TensorShape([1834, 64, 4]),
 TensorShape([1834, 64, 4]))

TensorShape([1834, 64, 4])

In [14]:
tf.expand_dims(tf.transpose(ds['left'], (0, 2, 1)), -1).shape

TensorShape([1834, 4, 64, 1])